### Full RAG Pipeline with Gemini (Google Generative AI)

In [ ]:
#!pip install langchain_community

In [3]:
import os
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS # A simple in-memory vector store
from langchain_core.documents import Document

In [ ]:
load_dotenv() # Load your API key from .env

# 1. Initialize embedding model
embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [ ]:
# --- Basic Embedding Usage ---
print("\n--- Basic Embedding Usage ---")
text = "LangChain makes building LLM applications easier."
embedding = embeddings_model.embed_query(text)
print(f"Embedding dimensions: {len(embedding)}")
print(f"First 5 dimensions: {embedding[:5]}")


### Using Embeddings with a Vector Store (Example: FAISS for RAG) ---

In [20]:
print("\n--- Using Embeddings with a Vector Store (for RAG) ---")
texts = ["LangChain integrates with Gemini.", "Embeddings power retrieval-augmented generation."]

# Generate embeddings
embeddings = embeddings_model.embed_documents(texts)

print("Embedding vector length:", len(embeddings[0]))
print("First 5 values of first embedding:", embeddings[0][:5])


--- Using Embeddings with a Vector Store (for RAG) ---
Embedding vector length: 768
First 5 values of first embedding: [0.057657234370708466, 0.010308874770998955, -0.044133931398391724, 0.009841084480285645, 0.0408058762550354]


In [ ]:
#!pip install faiss-cpu

In [11]:
# 2. Sample knowledge base
texts = [
    "LangChain is a framework for developing applications powered by LLMs.",
    "Gemini is a family of large language models developed by Google DeepMind.",
    "FAISS is a library for efficient similarity search and clustering of dense vectors."
]

In [12]:
# 3. Wrap into LangChain Documents
documents = [Document(page_content=txt) for txt in texts]

In [13]:
# 4. Create FAISS vector store
vectorstore = FAISS.from_documents(documents, embeddings_model)

In [ ]:
# Run a Similarity Search
query = "What is Gemini?"
results = vectorstore.similarity_search(query, k=2) # Get top 2 results

for idx, doc in enumerate(results):
    print(f"Match {idx+1}: {doc.page_content}")

Match 1: Gemini is a family of large language models developed by Google DeepMind.
Match 2: FAISS is a library for efficient similarity search and clustering of dense vectors.


### Use Gemini-Pro for Answer Generation

In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)

# Setup RAG pipeline using Retriever + LLM
rag_chain = RetrievalQA.from_chain_type(
    llm=llm, # Use the Gemini LLM
    retriever=vectorstore.as_retriever(), # Use the FAISS vector store as retriever
    return_source_documents=True # Return source documents for context
)

# Ask a question
query = "What is Gemini?"
response = rag_chain({"query": query})

# Print the answer
print("Answer:", response["result"])

Answer: Gemini is a family of large language models developed by Google DeepMind.


In [18]:
# Ask a question
query = "What is FAISS?"
response = rag_chain({"query": query})

# Print the answer
print("Answer:", response["result"])

Answer: FAISS is a library for efficient similarity search and clustering of dense vectors.


In [ ]:
# Ask a question - which is not in the knowledge base
query = "What is LLM?"
response = rag_chain({"query": query})

# Print the answer
print("Answer:", response["result"])

Answer: LLM stands for Large Language Model.
